# test_bdf demo

In this demo, we'll show off test_bdf

In [1]:
from IPython.display import HTML as html_print
from pyNastran.bdf.bdf import BDF, read_bdf, CaseControlDeck


In [2]:
model = BDF()

# add_grid(nid, xyz, cp=0, cd=0, ps='', seid=0)
model.add_grid(1, [0., 0., 0.])
model.add_grid(2, [1., 0., 0.])
model.add_grid(3, [1., 1., 0.])
model.add_grid(4, [0., 1., 0.])

eid = 10
pid = 100
mid = 1000
# add_cbar(eid, pid, nids, x, g0, offt='GGG', pa=0, pb=0, wa=None, wb=None)
model.add_cbar(eid,   pid, [1, 2], [0., 0., 1.], None, offt='GGG')
model.add_cbar(eid+1, pid, [2, 3], [0., 0., 1.], None, offt='GGG')
model.add_cbar(eid+2, pid, [3, 4], [0., 0., 1.], None, offt='GGG')
model.add_cbar(eid+3, pid, [4, 1], [0., 0., 1.], None, offt='GGG')

eid_cquad4 = 15
pid_pshell = 101
# add_pshell(pid, mid1=None, t=None, mid2=None, twelveIt3=1.0,
#            mid3=None, tst=0.833333, 
#            nsm=0.0, z1=None, z2=None, mid4=None)
model.add_pshell(pid_pshell, mid1=mid, t=0.1, mid2=mid, mid3=mid)
model.add_cquad4(eid_cquad4, pid_pshell, [1, 2, 3, 4])

dim = [3., 3., 1., 1.] # TODO: should be [1., 2., 3., 4.]
# add_pbarl(pid, mid, Type, dim, group='MSCBML0', nsm=0.0)
pbarl = model.add_pbarl(pid, mid, 'BOX', dim, nsm=0.0)
pbarl.validate()

E = 3.e7
G = None
nu = 0.3
mat = model.add_mat1(mid, E, G, nu)

spc_id = 1
nids = 1
# add_spc1(conid, components, nodes
model.add_spc1(spc_id, 123456, nids)


dresp_id = 100
label = 'resp1'
response_type = 'STRESS'
property_type = 'PSHELL'
pid = 3
atta = 9 # von mises upper surface stress
region = None
attb = None
atti = [pid_pshell]
# add_dresp1(dresp_id, label, response_type, property_type, region, atta, attb, atti)
model.add_dresp1(dresp_id, label, response_type, property_type, region, atta, attb, atti)

dresp_id += 1
atta = 17 # von mises lower surface stress
model.add_dresp1(dresp_id, label, response_type, property_type, region, atta, attb, atti)

# add_dconstr(oid, dresp_id, lid=-1e+20, uid=1e+20, lowfq=0.0, highfq=1e+20)
dconstr_id = 10000
model.add_dconstr(dconstr_id, dresp_id, lid=-35000., uid=35000.)

dresp_id += 1
dresp = model.add_dresp1(dresp_id, 'WT', 'WEIGHT', None, None, None, None, None)
dresp.validate()

oid = 1000
dvids = 1
coeffs = 1.
# add_dvprel1(oid, prop_type, pid, pname_fid, dvids, coeffs,
#             p_min=None, p_max=1e+20, c0=0.0)
model.add_dvprel1(oid, 'PSHELL', pid_pshell, 'T', dvids, coeffs)

# add_desvar(desvar_id, label, xinit, xlb=-1e+20, xub=1e+20,
#            delx=None, ddval=None)
model.add_desvar(1, 'DIM1', 0.1, xlb=1e-5)
model.add_desvar(2, 'DIM2', 0.2, xlb=1e-5)
model.add_desvar(3, 'DIM3', 0.3, xlb=1e-5)
model.add_desvar(4, 'DIM4', 0.4, xlb=1e-5)
model.add_desvar(5, 'DV5', 0.1, xlb=1e-5)

#model.add_dlink(6)

eid = 10 # TODO: remove
load_id = 1
# add_pload4(sid, eids, pressures, g1=None, g34=None,
#            cid=0, nvector=None, surf_or_line='SURF', line_load_dir='NORM')
pload4 = model.add_pload4(load_id, [eid_cquad4], [1., None, None, None], 
                          comment=' load')
#print(pload4.get_stats())

eid = 10 # TODO: should be 100
scale = 'LE' # TODO: should be 100.
# add_pload1(sid, eid, load_type, scale, x1, p1, x2=None, p2=None)
model.add_pload1(load_id, eid, 'FZ', scale, 0., 1.)  # TODO: change atti to None

# add_eigrl(sid, v1=None, v2=None, nd=None, msglvl=0, maxset=None, shfscl=None,
#           norm=None, options=None, values=None)
eigrl = model.add_eigrl(42, nd=42)

model.sol = 103  # start=103
cc = CaseControlDeck([
    'DESOBJ = 102',  # DRESP1
    'DESSUB = %s' % dconstr_id,  # DCONSTR
    'SUBCASE 1',
    '  METHOD = 42',  # TODO: remove
    '  LOAD = %s' % load_id,  # TODO: remove
    '  SPC = %s' % spc_id,
    '  TRIM = 42',  # TODO: add
    'ANALYSIS = SAERO',
])
#print(cc)
model.case_control_deck = cc
model.validate()

# rerun between each change
# 1. change SOL=103 -> SOL=144
model.sol = 144
# 2. add the trim in the case control deck
#help(model.add_trim)
mach = 0.8
q = 100.
labels = ['Z']
uxs = [2.5]
trim = model.add_trim(42, mach, q, labels, uxs, aeqr=1.0, trim_type=1)
print(trim)
# 3. add a trim card
# x. change to SOL=200
print(model.trims)
if 0:
    cref = bref = sref = 1.0
    model.add_aeros(cref, bref, sref)
    eid = 1
    pid = 1
    igroup = 1
    x12 = 1.0
    x43 = 2.0
    p1 = [0., 0., 0.]
    p4 = [0., 1., 0.]
    model.add_caero1(eid, pid, igroup, p1, x12, p4, x43, nchord=2, nspan=5)
    model.add_paero1(pid)
    model.add_spline1(eid=10, caero=eid, box1=1, box2=10, setg=50)
    model.add_set1(sid=50, ids=[11,12,13,14])
    model.add_grid(11, [0., 0., 0.])
    model.add_grid(12, [1., 0., 0.])
    model.add_grid(13, [1., 1., 0.])
    model.add_grid(14, [0., 1., 0.])
    del model.trims[42]
    labels = ['ANGLEA']
    uxs = [2.5]
    trim = model.add_trim(42, mach, q, labels, uxs, aeqr=1.0, trim_type=1)
    model.add_aestat(100,label='ANGLEA')
model.write_bdf('junk.bdf')
#!cat junk.bdf
print('----------------------------------------------------------------------------------------------------')

TRIM          42      .8    100.       Z     2.5

{42: TRIM          42      .8    100.       Z     2.5
}


----------------------------------------------------------------------------------------------------


In [3]:
from pyNastran.bdf.test.test_bdf import run_bdf as test_bdf
model.write_bdf('junk.bdf')
test_bdf('.', 'junk.bdf')

debug = False
bdf_model = junk.bdf


Traceback (most recent call last):
  File "C:\Users\sdoyle\Desktop\demo\pyNastran_1.4.0\pyNastran\bdf\test\test_bdf.py", line 462, in run_and_compare_fems
    fem2 = run_fem2(
  File "C:\Users\sdoyle\Desktop\demo\pyNastran_1.4.0\pyNastran\bdf\test\test_bdf.py", line 973, in run_fem2
    ierror = _validate_case_control(
  File "C:\Users\sdoyle\Desktop\demo\pyNastran_1.4.0\pyNastran\bdf\test\test_bdf.py", line 1036, in _validate_case_control
    ierror = check_case(
  File "C:\Users\sdoyle\Desktop\demo\pyNastran_1.4.0\pyNastran\bdf\test\test_bdf.py", line 1291, in check_case
    ierror = _check_static_aero_case(fem2, log, sol, subcase, ierror, nerrors)
  File "C:\Users\sdoyle\Desktop\demo\pyNastran_1.4.0\pyNastran\bdf\test\test_bdf.py", line 1368, in _check_static_aero_case
    ierror = stop_if_max_error(msg, RuntimeError, ierror, nerrors)
  File "C:\Users\sdoyle\Desktop\demo\pyNastran_1.4.0\pyNastran\bdf\test\test_bdf.py", line 1072, in stop_if_max_error
    raise error(msg)
RuntimeErro

RuntimeError: An AEROS card is required for STATIC AERO - SOL 144; AEROS=None